# Data Science with Python and Neo4j

## Packages You'll Need

```
pip install py2neo
pip install ipython-cypher
pip install pandas
pip install networkx
pip install python-igraph
```

## Contents

* py2neo
* ipython-cypher
  - pandas
  - NetworkX
* igraph
* graph3d

# py2neo

`py2neo` is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j. Install `py2neo` with `pip install py2neo`.

## Connect

Connect to Neo4j with the `Graph` class.

In [1]:
from py2neo import Graph
from scripts.vis import draw

graph = Graph()

graph.delete_all()

ImportError: No module named 'py2neo'

## Nodes

Create nodes with the `Node` class. The first argument is the node's label. The remaining arguments are an arbitrary amount of node properties or key-value pairs.

In [ ]:
from py2neo import Node

nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole, drew, mtdew, cokezero, coke, pepsi)

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}

draw(graph, options)

## Relationships

Create relationships between nodes with the `Relationship` class.

In [ ]:
from py2neo import Relationship

graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))
graph.create(Relationship(pepsi, "MAKES", mtdew))

draw(graph, options)

## Cypher

Retrieve Cypher query results with `Graph.cypher.execute`.

In [ ]:
query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS name, drink.name AS drink
"""

data = graph.cypher.execute(query)

data

Rows can be accessed by index.

In [ ]:
data[0]

Columns can be accessed by key or attribute.

In [ ]:
for row in data:
    print "{0} likes to drink {1}.".format(row["name"], row.drink)

## Parameterized Cypher

Pass parameters to Cypher queries by passing additional key-value arguments to `Graph.cypher.execute.` Parameters in Cypher are named and are wrapped in curly braces.

In [ ]:
query = """
MATCH (p:Person)-[:LIKES]->(drink:Drink)
WHERE p.name = {name}
RETURN p.name AS name, AVG(drink.calories) AS avg_calories
"""

data = graph.cypher.execute(query, name="Nicole")

data

# ipython-cypher

`ipython-cypher` exposes `%cypher` magic in Jupyter. Install `ipython-cypher` with `pip install ipython-cypher`.

In [ ]:
%load_ext cypher

## Cypher

`%cypher` is intended for one-line Cypher queries and `%%cypher` is intended for multi-line Cypher queries. Placing `%%cypher` above a Cypher query will display that query's results.

In [ ]:
%%cypher
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name, drink.name, drink.calories

## Pandas Data Frames

Cypher query results can be coerced to `pandas` data frames with the `get_dataframe` method. To assign Cypher query results to a variable, you need to use `%cypher` and separate lines with \\. You'll first need to install `pandas` with `pip install pandas`.

In [ ]:
results = %cypher MATCH (person)-[:LIKES]->(drink:Drink) \
                  RETURN person.name AS name, drink.name AS drink
    
df = results.get_dataframe()

df

In [ ]:
df.index

In [ ]:
df.iloc[[1]]

In [ ]:
df["name"]

# NetworkX Graphs

Cypher query results can be coerced to `NetworkX` MultiDiGraphs, graphs that permit multiple edges between nodes, with the `get_graph` method. You'll first need to install `NetworkX` with `pip install networkx`.

In [ ]:
import networkx as nx
%matplotlib inline

results = %cypher MATCH p = (:Person)-[:LIKES]->(:Drink) RETURN p

g = results.get_graph()

nx.draw(g)

In [ ]:
g.nodes(data=True)

In [ ]:
nx.degree(g)

# igraph

Cypher query results can be imported into `igraph` with `py2neo`. You'll need to install `igraph` with `pip install python-igraph`. Query results should be returned as edgelists, as `igraph` has a method for building an `igraph` object from a list of tuples representing edges between nodes.

In [ ]:
from py2neo import Graph as PGraph
from igraph import Graph as IGraph

neo4j = PGraph()

query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name, drink.name
"""

data = neo4j.cypher.execute(query)
data

In [ ]:
ig = IGraph.TupleList(data)

ig

In [ ]:
best = ig.vs.select(_degree = ig.maxdegree())["name"]
best

## graph3d

`graph3d`, originally named `igraph` but which I've renamed to avoid conflicts with the other `igraph`, will plot tuple lists as 3D graphs.

In [ ]:
import graph3d

graph3d.draw([(1, 2), (2, 3), (3, 4), (4, 1), (4, 5), (5, 2)])

We can visualize our Neo4j data by pulling out an edgelist with a Cypher query.

In [ ]:
from py2neo import Graph

graph = Graph()

data = graph.cypher.execute("MATCH (n)-->(m) RETURN ID(n), ID(m)")
data = [tuple(x) for x in data]

graph3d.draw(data)